In [1]:
%reload_ext autotime
import geopandas as gpd
import pandas as pd
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error
from coastsat import SDS_transects
pd.options.plotting.backend = "plotly"

In [2]:
# Transects, origin is landward. Has beach_slope
transects = gpd.read_file("transects_extended.geojson")
transects.set_index("id", inplace=True)
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.297976,1371.0,1371.0,0.203601,5.662483,50.986351,7.140473,199.127124,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.011742,1371.0,1371.0,0.000671,3.998349,30.150238,5.490923,165.933848,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.048766,1371.0,1371.0,0.008370,4.829729,41.363000,6.431407,161.418975,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [3]:
vos_files = pd.Series(
    sorted(glob("csv_run7/*/time_series_tidally_corrected.csv"))
)
vos_files = vos_files[~vos_files.str.contains("nzd")]
vos_files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
3066    csv_run7/usa_HI_0053/time_series_tidally_corre...
3067    csv_run7/usa_HI_0054/time_series_tidally_corre...
3068    csv_run7/usa_HI_0055/time_series_tidally_corre...
3069    csv_run7/usa_HI_0056/time_series_tidally_corre...
3070    csv_run7/usa_HI_0057/time_series_tidally_corre...
Length: 2511, dtype: object

In [4]:
my_nz_files = pd.Series(
    sorted(glob("data/*/transect_time_series_tidally_corrected.csv"))
)
my_nz_files

0      data/nzd0001/transect_time_series_tidally_corr...
1      data/nzd0002/transect_time_series_tidally_corr...
2      data/nzd0003/transect_time_series_tidally_corr...
3      data/nzd0004/transect_time_series_tidally_corr...
4      data/nzd0005/transect_time_series_tidally_corr...
                             ...                        
557    data/nzd0559/transect_time_series_tidally_corr...
558    data/nzd0560/transect_time_series_tidally_corr...
559    data/nzd0561/transect_time_series_tidally_corr...
560    data/nzd0562/transect_time_series_tidally_corr...
561    data/nzd0563/transect_time_series_tidally_corr...
Length: 562, dtype: object

In [5]:
sar_files = pd.Series(sorted(glob("data/sar*/transect_time_series.csv")))
sar_files

0       data/sar0001/transect_time_series.csv
1       data/sar0002/transect_time_series.csv
2       data/sar0003/transect_time_series.csv
3       data/sar0004/transect_time_series.csv
4       data/sar0005/transect_time_series.csv
                        ...                  
2536    data/sar2537/transect_time_series.csv
2537    data/sar2538/transect_time_series.csv
2538    data/sar2539/transect_time_series.csv
2539    data/sar2540/transect_time_series.csv
2540    data/sar2541/transect_time_series.csv
Length: 2541, dtype: object

In [6]:
files = pd.concat([vos_files, my_nz_files, sar_files])
files

0       csv_run7/aus0001/time_series_tidally_corrected...
1       csv_run7/aus0002/time_series_tidally_corrected...
2       csv_run7/aus0003/time_series_tidally_corrected...
3       csv_run7/aus0004/time_series_tidally_corrected...
4       csv_run7/aus0005/time_series_tidally_corrected...
                              ...                        
2536                data/sar2537/transect_time_series.csv
2537                data/sar2538/transect_time_series.csv
2538                data/sar2539/transect_time_series.csv
2539                data/sar2540/transect_time_series.csv
2540                data/sar2541/transect_time_series.csv
Length: 5614, dtype: object

In [7]:
f = files[files.str.contains("sar1026")].iloc[0]
# despiked_filename = f.replace(".csv", "_tidally_corrected.csv")
df = pd.read_csv(f)
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
display(df.columns)
import matplotlib.pyplot as plt

transect_id = "sar1026-0007"


def custom_mean(window):
    return window[window.between(window.quantile(0.25), window.quantile(0.75))].mean()


pd.DataFrame(
    {
        "raw": df[transect_id],
        "rolling 90d mean": df[transect_id].rolling("90d", min_periods=1).mean(),
        "rolling 180d mean": df[transect_id].rolling("180d", min_periods=1).mean(),
        "rolling 90d custom mean": df[transect_id]
        .rolling("90d", min_periods=1)
        .apply(custom_mean),
        "rolling 180d custom mean": df[transect_id]
        .rolling("180d", min_periods=1)
        .apply(custom_mean),
        # "rolling 365d": df[transect_id].rolling("365d", min_periods=1).mean(),
    },
    index=df.index,
).plot()

Index(['satname', 'sar1026-0000', 'sar1026-0001', 'sar1026-0002',
       'sar1026-0003', 'sar1026-0004', 'sar1026-0005', 'sar1026-0006',
       'sar1026-0007'],
      dtype='object')

In [8]:
df = pd.read_csv("data/sar0939/transect_time_series.csv")
df.dates = pd.to_datetime(df.dates)
df.set_index("dates", inplace=True)
(df["sar0939-0000"] - 93).plot()

In [9]:
def despike(chainage, threshold=40):
    chainage = chainage.dropna()
    chainage, dates = SDS_transects.identify_outliers(
        chainage.tolist(), chainage.index.tolist(), threshold
    )
    return pd.Series(chainage, index=dates)


def get_trends(f):
    df = pd.read_csv(f)
    try:
        df.dates = pd.to_datetime(df.dates)
    except:
        print(f)
    if "sar" in f:
        smoothed_filename = f.replace(".csv", "_smoothed.csv")
        try:
            raise
            df = pd.read_csv(smoothed_filename)
            df.dates = pd.to_datetime(df.dates)
        except:
            df.dates = pd.to_datetime(df.dates)
            df.set_index("dates", inplace=True)
            satname = df.satname
            df = df.drop(columns="satname").apply(despike, axis=0)
            df["satname"] = satname
            df.reset_index(names="dates").to_csv(
                f.replace(".csv", "_despiked.csv"), index=False
            )
            for transect_id in df.drop(columns="satname").columns:
                df[transect_id] = df[transect_id].rolling("180d", min_periods=1).mean()
            df.reset_index(names="dates", inplace=True)
            df.to_csv(f.replace(".csv", "_smoothed.csv"), index=False)
    df.index = (df.dates - df.dates.min()).dt.days / 365.25
    df.drop(columns=["dates", "satname", "Unnamed: 0"], inplace=True, errors="ignore")
    trends = []
    for transect_id in df.columns:
        sub_df = df[transect_id].dropna()
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1, 1)
        y = sub_df
        linear_model = LinearRegression().fit(x, y)
        pred = linear_model.predict(x)
        trends.append(
            {
                "transect_id": transect_id,
                "trend": linear_model.coef_[0],
                "intercept": linear_model.intercept_,
                "n_points": len(df[transect_id]),
                "n_points_nonan": len(sub_df),
                "r2_score": r2_score(y, pred),
                "mae": mean_absolute_error(y, pred),
                "mse": mean_squared_error(y, pred),
                "rmse": root_mean_squared_error(y, pred),
            }
        )
    return pd.DataFrame(trends)


# trends = get_trends(sar_files.iloc[-1]).set_index("transect_id")
trends = pd.concat(process_map(get_trends, files)).set_index("transect_id")
len(trends)

/tmp/ipykernel_1549383/4051723390.py:62: TqdmWarning:

Iterable length 5614 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.



  0%|          | 0/5614 [00:00<?, ?it/s]

*

*

*

...

*

*

*

*

141767

In [10]:
trends[trends.n_points_nonan > 10].sort_values("r2_score")

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
transect_id,,,,,,,,
chi0380-0121,-0.000004,219.970391,808,471,2.868705e-12,14.791994,350.500589,18.721661
jap0061-0003,-0.000004,163.091318,547,396,4.766876e-11,4.395403,36.736665,6.061078
usa_HI_0023-0012,0.000029,228.048992,447,310,5.878720e-11,26.568889,891.677012,29.860961
nzd0454-0235,-0.000014,339.463241,356,234,1.108120e-10,7.395218,92.562606,9.620946
mex0141-0116,0.000015,229.483758,609,546,1.189572e-10,10.931806,179.210279,13.386944
...,...,...,...,...,...,...,...,...
sar0812-0006,-6.589307,357.224708,505,503,9.898876e-01,6.341615,59.706639,7.727007
sar0812-0007,-6.813410,364.364517,505,497,9.904291e-01,6.040046,56.876702,7.541664
sar0811-0004,-7.113417,395.283787,561,547,9.914148e-01,6.402822,55.450602,7.446516


In [11]:
trends[trends.index.str.startswith("sar")].describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,19763.000000,19763.000000,19763.000000,19763.000000,1.976300e+04,19763.000000,19763.000000,19763.000000
mean,0.014860,227.003780,972.607752,970.975560,1.585299e-01,5.956841,294.648890,8.025057
std,0.894668,95.680419,308.804627,309.544986,1.967021e-01,11.681717,1820.485492,15.174288
min,-21.512526,-144.215125,187.000000,8.000000,2.320397e-09,0.000000,0.000000,0.000000
25%,-0.112449,185.251006,694.000000,693.000000,1.665420e-02,2.067728,7.187000,2.680858
50%,-0.007568,203.555695,930.000000,929.000000,7.568419e-02,2.794240,13.148723,3.626117
75%,0.085343,227.113848,1299.000000,1298.000000,2.272271e-01,4.532811,35.362096,5.946604
max,13.252247,1008.632670,1453.000000,1453.000000,1.000000e+00,284.061345,87118.699682,295.158770


In [12]:
trends.describe()

,trend,intercept,n_points,n_points_nonan,r2_score,mae,mse,rmse
count,141767.000000,141767.000000,141767.000000,141767.000000,1.417670e+05,141767.000000,141767.000000,141767.000000
mean,0.193238,235.453775,683.796434,572.686591,1.788681e-01,11.934687,348.573754,15.173524
std,1.839130,97.505803,319.862116,301.875820,2.200773e-01,8.696690,945.672478,10.878362
min,-61.628890,-2782.367945,62.000000,8.000000,2.868705e-12,0.000000,0.000000,0.000000
25%,-0.242266,182.563712,452.000000,342.000000,1.774142e-02,7.477627,92.591695,9.622458
50%,0.069375,213.716670,625.000000,524.000000,8.375681e-02,10.314210,170.818998,13.069774
75%,0.518100,296.845632,857.000000,708.000000,2.610257e-01,14.015006,313.777226,17.713758
max,90.298981,2327.755712,1767.000000,1523.000000,1.000000e+00,284.061345,87118.699682,295.158770


In [13]:
(transects.trend - trends.trend).describe()

count    141789.000000
mean          0.000208
std           0.013009
min          -0.464650
25%           0.000000
50%           0.000000
75%           0.000000
max           0.608063
Name: trend, dtype: float64

In [14]:
transects.update(trends.drop_duplicates())

In [15]:
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.297976,1371.0,1371.0,0.203601,5.662483,50.986351,7.140473,199.127124,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.011742,1371.0,1371.0,0.000671,3.998349,30.150238,5.490923,165.933848,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.048766,1371.0,1371.0,0.008370,4.829729,41.363000,6.431407,161.418975,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [16]:
trends.columns, transects.columns, trends.columns.isin(transects.columns)

(Index(['trend', 'intercept', 'n_points', 'n_points_nonan', 'r2_score', 'mae',
        'mse', 'rmse'],
       dtype='object'),
 Index(['site_id', 'orientation', 'along_dist', 'along_dist_norm',
        'beach_slope', 'cil', 'ciu', 'trend', 'n_points', 'n_points_nonan',
        'r2_score', 'mae', 'mse', 'rmse', 'intercept', 'ERODIBILITY',
        'geometry'],
       dtype='object'),
 array([ True,  True,  True,  True,  True,  True,  True,  True]))

In [17]:
transects = transects.join(trends.loc[:, ~trends.columns.isin(transects.columns)])
transects

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
aus0001-0000,aus0001,104.347648,0.000000,0.000000,0.085,0.0545,0.2000,-1.441081,767.0,428.0,0.168420,28.102591,1263.560863,35.546601,179.085729,None,"LINESTRING (153.26555 -24.7007, 153.26938 -24...."
aus0001-0001,aus0001,93.495734,98.408334,0.002935,0.050,0.0387,0.0640,-1.037105,767.0,569.0,0.097874,25.419324,1033.770813,32.152306,212.247788,None,"LINESTRING (153.26525 -24.7019, 153.2692 -24.7..."
aus0001-0002,aus0001,82.069341,198.408334,0.005918,0.050,0.0428,0.0647,-0.680019,767.0,588.0,0.053927,22.632907,838.007507,28.948359,205.106151,None,"LINESTRING (153.26539 -24.70316, 153.26931 -24..."
aus0001-0003,aus0001,81.192757,298.402523,0.008900,0.055,0.0480,0.0659,-0.405198,767.0,598.0,0.023412,20.749758,698.653187,26.432048,191.745881,None,"LINESTRING (153.26555 -24.70408, 153.26945 -24..."
aus0001-0004,aus0001,81.065473,398.402523,0.011882,0.075,0.0614,0.0922,-0.090025,767.0,608.0,0.001277,19.889328,655.810616,25.608800,175.092121,None,"LINESTRING (153.2657 -24.70497, 153.26961 -24...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.297976,1371.0,1371.0,0.203601,5.662483,50.986351,7.140473,199.127124,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.011742,1371.0,1371.0,0.000671,3.998349,30.150238,5.490923,165.933848,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.048766,1371.0,1371.0,0.008370,4.829729,41.363000,6.431407,161.418975,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [18]:
transects[transects.site_id.str.startswith("sar") & ~transects.trend.isna()]

,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
id,,,,,,,,,,,,,,,,,
sar0001-0000,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.209107,671.0,667.0,0.003085,34.337637,2024.899412,44.998882,135.992639,Medium,"LINESTRING (8.40852 38.86175, 8.40882 38.86535)"
sar0001-0001,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.168722,671.0,667.0,0.008700,11.931958,460.730301,21.464629,197.877604,Medium,"LINESTRING (8.4084 38.86162, 8.41092 38.86464)"
sar0001-0002,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.012318,671.0,669.0,0.000413,5.801157,51.994338,7.210710,206.410952,Medium,"LINESTRING (8.40893 38.86153, 8.41236 38.86393)"
sar0001-0003,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,-0.043548,671.0,669.0,0.016188,3.202754,16.331152,4.041182,239.500879,Medium,"LINESTRING (8.40904 38.86129, 8.41333 38.8626)"
sar0001-0004,sar0001,NaN,NaN,NaN,NaN,NaN,NaN,0.011825,671.0,669.0,0.001501,2.692829,13.178444,3.630213,258.136097,Medium,"LINESTRING (8.40904 38.86176, 8.4133 38.86037)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sar2541-0000,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,-0.297976,1371.0,1371.0,0.203601,5.662483,50.986351,7.140473,199.127124,High,"LINESTRING (8.85399 38.88006, 8.85388 38.87736)"
sar2541-0001,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.011742,1371.0,1371.0,0.000671,3.998349,30.150238,5.490923,165.933848,High,"LINESTRING (8.85428 38.88005, 8.85417 38.87735)"
sar2541-0002,sar2541,NaN,NaN,NaN,NaN,NaN,NaN,0.048766,1371.0,1371.0,0.008370,4.829729,41.363000,6.431407,161.418975,High,"LINESTRING (8.85502 38.87993, 8.85398 38.87735)"


In [19]:
transects.drop_duplicates().to_file("transects_extended.geojson", driver="GeoJSON")